In [28]:
import cv2
import util

In [29]:
#pyautogui  also contains modules to control mmouse but pynput provides better methods for mouse operations
import pyautogui
from pynput.mouse import Button,Controller
mouse=Controller()

In [30]:
import random
import mediapipe as mp
#helps to give landmarks to fingers and identify the required marks

In [31]:
screen_width ,screen_height=pyautogui.size()
mpHands=mp.solutions.hands
hands=mpHands.Hands(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.7,
    #track if visibility is above  70%
    min_tracking_confidence=0.7,
    #max hands in a frame should be 1
    max_num_hands=1
)

In [32]:
def find_finger_tip(processed):
    if processed.multi_hand_landmarks:
        hand_landmarks=processed.multi_hand_landmarks[0]
        return hand_landmarks.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP]
    return None

In [33]:
def move_mouse(index_finger_tip):
    if index_finger_tip is not None:
        x=int(index_finger_tip.x*screen_width)
        y=int(index_finger_tip.y*screen_height)
        pyautogui.moveTo(x,y)

In [34]:
def is_left_click(landmarks_list,thumb_index_dist):
    return (util.get_angle(landmarks_list[5],landmarks_list[6],landmarks_list[8])<50 and 
            util.get_angle(landmarks_list[9] ,landmarks_list[10],landmarks_list[12])>90 and thumb_index_dist>50)
def is_right_click(landmarks_list,thumb_index_dist):
    return (util.get_angle(landmarks_list[9],landmarks_list[10],landmarks_list[12])<50 and 
            util.get_angle(landmarks_list[5] ,landmarks_list[6],landmarks_list[8])>90 and thumb_index_dist>50)
def is_double_click(landmarks_list,thumb_index_dist):
    return (util.get_angle(landmarks_list[5],landmarks_list[6],landmarks_list[8])<50 and 
            util.get_angle(landmarks_list[9] ,landmarks_list[10],landmarks_list[12])<50 and thumb_index_dist>50)
def is_screenshot(landmarks_list,thumb_index_dist):
    return (util.get_angle(landmarks_list[5],landmarks_list[6],landmarks_list[8])<50 and 
            util.get_angle(landmarks_list[9] ,landmarks_list[10],landmarks_list[12])<50 and thumb_index_dist<50)


In [35]:
def detect_gestures(frame,landmarks_list,processed):
    if len(landmarks_list)>=21:
        index_finger_tip=find_finger_tip(processed)
        thumb_index_dist=util.get_distance((landmarks_list[4],landmarks_list[5]))
        if thumb_index_dist<50 and util.get_angle(landmarks_list[5],landmarks_list[6],landmarks_list[8])>90:
            move_mouse(index_finger_tip)
        #now code for left click operation method is_left_click
        elif is_left_click(landmarks_list,thumb_index_dist):
            mouse.press(Button.left)
            cv2.putText(frame,"Left Click",(50 ,50), cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
            
         #right click   
        elif is_right_click(landmarks_list,thumb_index_dist):
            mouse.press(Button.right)
            mouse.release(Button.right)
            cv2.putText(frame,"Right Click",(50 ,50), cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
            
        #double click    
        elif is_double_click(landmarks_list,thumb_index_dist):
            pyautogui.doubleClick()
            cv2.putText(frame,"Double Click",(50 ,50), cv2.FONT_HERSHEY_COMPLEX,1,(255,255,0),2)
            
            
        #screenshot
        elif is_screenshot(landmarks_list,thumb_index_dist):
            im1=pyautogui.screenshot()
            label=random.randint(1,1000)
            im1.save(f'my_screenshot_{label}.png')
            cv2.putText(frame,"Screenshot taken",(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,0),2)
            
            

In [36]:
def main():
    capture = cv2.VideoCapture(0)
    draw=mp.solutions.drawing_utils
    try:
        while capture.isOpened():
            ret, frame = capture.read()
            if not ret:
                break
            frame = cv2.flip(frame, 1)
            frameRGB=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
            #it will process the frame and identify all landmarks
            processed=hands.process(frameRGB)
            landmarks_list=[]
            if processed.multi_hand_landmarks:
                hand_landmarks=processed.multi_hand_landmarks[0]
                draw.draw_landmarks(frame,hand_landmarks,mpHands.HAND_CONNECTIONS)
                for lm in hand_landmarks.landmark:
                    landmarks_list.append((lm.x,lm.y))
                detect_gestures(frame,landmarks_list,processed)
                #print(landmarks_list)
            cv2.imshow('frame', frame)
            if cv2.waitKey(1) & 0xFF == ord('c'):
                break
    finally:
        capture.release()
        cv2.destroyAllWindows()

if __name__ == '__main__':
    main()